In [ ]:
import torch
import numpy as np

import random
import copy
import os
import pickle

import cl_gym as cl
from metrics import MetricCollector2, FairMetricCollector
from configs import make_params

std_datasets = ["MNIST", "FashionMNIST", "CIFAR10", "CIFAR100"]

In [ ]:
def load(dataset, seed, epoch, lr, tau, alpha, lmbd, method, verbose=0):
    log_dir = f"scripts_output/dataset={dataset}"
    target_dir = f"seed={seed}_epoch={epoch}_lr={lr}"
    log_dir = os.path.join(log_dir, method)
    if method in ['FSW', "GSS"]:
        target_dir+=f"_tau={tau}_alpha={alpha}"
        if alpha != 0 and dataset not in std_datasets:
            target_dir+=f"_lmbd={lmbd}_lmbdold={0.0}"

    if not target_dir in os.listdir(log_dir):
        for dir in os.listdir(log_dir):
            if target_dir in dir:
                target_dir = dir
                break

    # print(f"{target_dir=}")
    log_dir = os.path.join(log_dir, target_dir)
    # print(f"{log_dir=}")
    out = os.path.join(log_dir, "log.out")
    err = os.path.join(log_dir, "log.err")
    if os.path.exists(err):
        with open(err, "r") as f:
            lines = f.readlines()
        if len(lines):
            if verbose:
                print(f"Error in {target_dir} - error during running")
                for line in lines:
                    print(line)
            return False
    else:
        if verbose:
            print(f"error in {target_dir} - not exists")
        return False
    return target_dir

def check_running(dataset, target_dir, method):
    log_dir = f"scripts_output/dataset={dataset}"
    log_dir = os.path.join(log_dir, method)
    log_dir = os.path.join(log_dir, target_dir)        
    out = os.path.join(log_dir, "log.out")
    if os.path.exists(out):
        with open(out, "r") as f:
            lines = f.readlines()
    if len(lines) == 0:
        return True
    else:
        return False

def load_log(dataset, target_dir, method):
    log_dir = f"scripts_output/dataset={dataset}"
    log_dir = os.path.join(log_dir, method)
    log_dir = os.path.join(log_dir, target_dir)      
    # print(f"load_log_log_{log_dir=}")  
    out = os.path.join(log_dir, "log.out")
    acc, fairness = 0, 0
    if os.path.exists(out):
        with open(out, "r") as f:
            lines = f.readlines()
        if "accuracy" in lines[-2]:
            acc = float(lines[-2].strip().split(":")[-1])
        elif "accuracy" in lines[-3]:
            acc = float(lines[-3].strip().split(":")[-1])
        if "fairness" in lines[-1]:
            fairness = float(lines[-1].strip().split(":")[-1])
        elif "fairness" in lines[-2]:
            fairness = float(lines[-2].strip().split(":")[-1])
    else:
        print(f"{out=} not exists")
    if acc * fairness == 0:
        print(f"{out=} : wrong output")
    return acc, fairness

def load_metrics(dataset, target_dir, method, validation=False):
    metrics_dir = f"outputs/dataset={dataset}"
    metrics_dir = os.path.join(metrics_dir, method)

    metrics_dir = os.path.join(metrics_dir, target_dir)
    if not os.path.exists(metrics_dir):
        metrics_dir = metrics_dir.replace("lmbd=", "lmbd_")
        metrics_dir = metrics_dir.replace("lmbdold=", "lmbdold_")
    out = os.path.join(metrics_dir, "plots/output.txt")

    if validation:
        if os.path.exists(out):
            with open(out, "r") as f:
                lines = f.readlines()
            for i, line in enumerate(lines):
                if i == 0:
                    continue
                if "acc" in line:
                    acc = float(lines[i+1].strip())
        else:
            print(f"{out=} not exists")
        if acc != load_log(dataset, target_dir, method)[0]:
            print(acc)
            print(load_log(dataset, target_dir, method)[0])
            raise ValueError
    
    metrics_dir = os.path.join(metrics_dir, "metrics/metrics.pickle")
    with open(metrics_dir, "rb") as f:
        metric_manager_callback = pickle.load(f)
    return metric_manager_callback

In [ ]:
def get_best(dataset, seed_range, epoch, lr_range, tau_range, alpha_range, lambda_range, \
             method, verbose=0):
    acc_list = list()
    fair_list = list()
    info_list = list()
    fair_metric = "multiclass_eo"
    if dataset in std_datasets:
        lambda_range = [0]
        fair_metric = "std"
    # MNIST
    for tau in tau_range:
        for lr in lr_range:
            for alpha in alpha_range:
                for lmbd in lambda_range:
                    cnt = 0
                    acc_sum = 0
                    fair_sum = 0
                    info = f"tau={tau}/lr_{lr}/alpha_{alpha}"
                    # print(f"{info=}")
                    avail_seed = copy.deepcopy(seed_range)
                    for seed in seed_range:
                        target_dir = load(dataset, seed, epoch, lr, tau, alpha, lmbd, method)
                        # print(f"{target_dir=}")
                        if not target_dir or check_running(dataset, target_dir, method):
                            avail_seed.remove(seed)
                            # print(seed)
                            # print(target_dir)
                            continue
                        acc, fair = load_log(dataset, target_dir, method)
                        if acc*fair == 0:
                            avail_seed.remove(seed)
                            # print(seed)
                            # print(target_dir)
                            continue
                        mmc = load_metrics(dataset, target_dir, method)
                        
                        acc = np.mean(mmc.meters['accuracy'].compute_overall())
                        fair = np.mean(mmc.meters[fair_metric].compute_overall())
                        acc_sum+=acc
                        fair_sum+=fair
                        cnt+=1
                    info = f"tau={tau}/lr_{lr}/alpha_{alpha}({cnt=}, {avail_seed})"
                    if cnt == 0:
                        print(f"{info=}: check if boom?")
                        continue

                    info_list.append(info)
                    acc_list.append(acc_sum/cnt)
                    fair_list.append(fair_sum/cnt)
                    out = f"{info}\n{acc_sum/cnt}\n{fair_sum/cnt}"
                    if verbose:
                        print(out)
    if verbose:
        print()
    
    integrated_score = [e - 2*fair_list[i] for i, e in enumerate(acc_list)]
    idx = integrated_score.index(max(integrated_score))
    print(f"{info_list[idx]}")
    accuracy = acc_list[idx]
    fairness = fair_list[idx]

    if accuracy > 1:
        accuracy /= 100
    print(f"acc:{accuracy}")
    print(f"fair:{fairness}")
    return info_list, acc_list, fair_list


In [ ]:
import matplotlib.pyplot as plt

def plot(acc_list, fair_list, marker = "o"):
    plt.scatter(acc_list, fair_list, marker=marker, s = 10)
    plt.xlabel('acc')
    plt.ylabel('fairness')

In [ ]:
def display(dataset, epoch, size = 20):
    print("joint")
    tau_range = [0.0]
    # lr_range = [0.01, 0.001]
    lr_range = [0.01]
    alpha_range = [0.0]
    lambda_range = [0.0]
    seed_range = [0, 1, 2, 3, 4]

    joint_info_list, joint_acc_list, joint_fair_list = get_best(dataset, seed_range, epoch, lr_range, tau_range, alpha_range, lambda_range, "joint")
    plt.scatter(joint_acc_list, joint_fair_list, marker='o', s = size)

    print("finetune")
    tau_range = [0.0]
    alpha_range = [0.0]
    lambda_range = [0.0]
    seed_range = [0, 1, 2, 3, 4]
    finetune_info_list, finetune_acc_list, finetune_fair_list = get_best(dataset, seed_range, epoch, lr_range, tau_range, alpha_range, lambda_range, "finetune")
    # plt.scatter(finetune_acc_list, finetune_fair_list, marker='x', s = size)


    print("vanilla")
    tau_range = [1.0, 5.0, 10.0]
    alpha_range = [0.0]
    lambda_range = [1.0, 5.0, 10.0]
    seed_range = [0, 1, 2, 3, 4]

    base_info_list, base_acc_list, base_fair_list = get_best(dataset, seed_range, epoch, lr_range, tau_range, alpha_range, lambda_range, "vanilla")
    plt.scatter(base_acc_list, base_fair_list, marker='v', s = size)

    print("FSW")
    tau_range = [1.0, 5.0, 10.0]
    alpha_range = [0.0005, 0.001, 0.002, 0.005, 0.01, 0.02]
    lambda_range = [1.0, 5.0, 10.0]
    seed_range = [0, 1, 2, 3, 4]

    fss_info_list, fss_acc_list, fss_fair_list = get_best(dataset, seed_range, epoch, lr_range, tau_range, alpha_range, lambda_range, "FSW")
    for i, e in enumerate(fss_info_list):
        print(e, fss_acc_list[i], fss_fair_list[i])
    plt.scatter(fss_acc_list, fss_fair_list, marker='x', s = size)

    target_baseline = [e - 2*base_fair_list[i] for i, e in enumerate(base_acc_list)]
    idx = target_baseline.index(max(target_baseline))
    base_fair_max = base_fair_list[idx]
    base_acc_min = base_acc_list[idx]
    plt.xlim([base_acc_min-5, 100])
    if dataset != "BiasedMNIST":
        plt.ylim([0, base_fair_max+5])
    else:
        plt.ylim([0, base_fair_max+0.15])

    plt.xlabel('acc')
    if dataset in std_datasets:
        plt.ylabel('std')
    else:
        plt.ylabel('eo')

In [ ]:
dataset = "MNIST"
epoch = 5
display(dataset, epoch)

In [ ]:
dataset = "MNIST"
epoch = 5
display(dataset, epoch)

In [ ]:
dataset = "MNIST"
epoch = 15
display(dataset, epoch)

In [ ]:
print("gss")
dataset = "MNIST"
epoch = 1
tau_range = [1.0, 5.0, 10.0]
alpha_range = [0.0]
lr_range = [0.01, 0.001]
lambda_range = [0.0]
seed_range = [0, 1, 2]

base_info_list, base_acc_list, base_fair_list = get_best(dataset, seed_range, epoch, lr_range, tau_range, alpha_range, lambda_range)
plt.scatter(base_acc_list, base_fair_list, marker='v', s = 10)


In [ ]:
tau_range = [1.0, 5.0, 10.0]
lr_range = [0.01, 0.001]
alpha_range = [0.0005, 0.001, 0.002, 0.005, 0.01, 0.02]
lambda_range = [1.0, 5.0, 10.0]
seed_range = [0, 1, 2, 3, 4]


info_list, acc_list, fair_list = get_best(dataset, seed_range, epoch, lr_range, tau_range, alpha_range, lambda_range)
plot(acc_list, fair_list)

In [ ]:
tau_range = [1.0, 5.0, 10.0]
lr_range = [0.01, 0.001]
alpha_range = [0.0]
lambda_range = [1.0, 5.0, 10.0]
seed_range = [0, 1, 2, 3, 4]

info_list, acc_list, fair_list = get_best(dataset, seed_range, epoch, lr_range, tau_range, alpha_range, lambda_range)
plot(acc_list, fair_list)

In [ ]:
tau_range = [0.0]
lr_range = [0.01, 0.001]
alpha_range = [0.0]
seed_range = [0, 1, 2, 3, 4]

info_list, acc_list, fair_list = get_best(dataset, seed_range, epoch, lr_range, tau_range, alpha_range, lambda_range, joint=True)
plot(acc_list, fair_list)

In [ ]:
tau_range = [1.0, 5.0, 10.0]
lr_range = [0.01, 0.001]
alpha_range = [0.0005, 0.001, 0.002, 0.005, 0.01, 0.02]
lambda_range = [1.0, 5.0, 10.0]
seed_range = [0, 1, 2, 3, 4]

dataset = "FashionMNIST"
epoch = 1
lmbd = 1.0

info_list, acc_list, fair_list = get_best(dataset, seed_range, epoch, lr_range, tau_range, alpha_range, lambda_range)

In [ ]:
tau_range = [1.0, 5.0, 10.0]
lr_range = [0.01, 0.001]
alpha_range = [0.0]
lambda_range = [1.0, 5.0, 10.0]
seed_range = [0, 1, 2, 3, 4]

dataset = "FashionMNIST"
epoch = 1
lmbd = 1.0

info_list, acc_list, fair_list = get_best(dataset, seed_range, epoch, lr_range, tau_range, alpha_range, lambda_range)